In [1]:
import numpy as np

In [66]:
class Layer():

    def __init__(self , output_size , activation_function):
        self.input_size = None
        self.output_size = output_size
        self.input = None
        self.output = np.zeros(output_size)
        self.weights = []
        self.bias = []
        self.activation_function = None
        self.activation_function_derivative = None

        if activation_function == "relu":
            self.activation_function = self.relu
        elif activation_function == "sigmoid":
            self.activation_function = self.sigmoid
        elif activation_function == "tanh":
            self.activation_function = self.tanh

        if activation_function == "relu":
            self.activation_function_derivative = self.relu_derivative
        elif activation_function == "sigmoid":
            self.activation_function_derivative = self.sigmoid_derivative
        elif activation_function == "tanh":
            self.activation_function_derivative = self.tanh_derivative


    def setup(self ,input_size):
        self.input_size = input_size
        print(self.input_size)
        self.weights = np.random.rand(self.input_size, self.output_size)
        self.bias = np.random.rand(1, self.output_size)


    def forward(self, input):
        self.input = input
        self.output = np.dot(input, self.weights) + self.bias
        return  self.activation_function(self.output)


    def backward(self, output_gradient, learning_rate):
        output_gradient = output_gradient * self.activation_function_derivative(self.output)
        weights_gradient = np.dot(self.input.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        backward = np.dot(output_gradient, self.weights.T)
        backward = backward
        return backward

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def tanh_derivative(self, x):
        return 1 - x**2


class softmax_layer():

    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        self.input = input
        self.output = np.exp(input) / np.sum(np.exp(input), axis=1, keepdims=True)
        return self.output



class Cross_Entropy_layer():

    def __init__(self):
        self.target = None
        self.input = None
        self.output = None
        self.loss = None

    def forward(self, input, target ):

        self.output = self.cross_entropy(input, target)
        return self.output

    def backward(self):

        return self.output - self.target

    def cross_entropy(self, input, target):

        input = np.clip(input, 1e-15, 1 - 1e-15)
        self.loss = -np.sum(target * np.log(input)) / input.shape[0]
        return self.loss




class Network():

  def __init__(self , input_size ):
    self.layers = []
    self.in_dim =  input_size
    self.out_dim = None
    self.soft_max_layer = None
    self.cross_entropy = None
    self.input = None
    self.target = None

  def add(self, layer):
    self.layers.append(layer)

  def setup(self , input , target):
    self.input = input
    self.target = target
    self.out_dim = self.layers[-1].output_size
    self.layers[0].setup(self.in_dim)
    for i in range(1, len(self.layers)):
      self.layers[i].setup(self.layers[i-1].output_size)
    self.soft_max_layer = softmax_layer()
    self.cross_entropy = Cross_Entropy_layer(self.input , self.target)




  def forward(self  ):

    for i in range(1, len(self.layers)):

      self.layers[i].input = self.layers[i-1].forward(input)
      self.layers[i].forward()
    self.soft_max_layer.forward(self.layers[-1].output)
    self.cross_entropy.forward(self.soft_max_layer.output, target)
    print("Loss = " , self.cross_entropy.loss)
    return self.soft_max_layer.output














In [69]:
# Create a simple dataset (1 sample, 4 features, 3 output classes)
np.random.seed(0)
X = np.random.rand(1, 4)
y = np.array([[1, 0, 0]])  # class 0 is correct (one-hot)

NoneTypenet = Network(input_size=4)
net.add(Layer(output_size=5, activation_function='relu'))  # hidden layer
net.add(Layer(output_size=3, activation_function='sigmoid'))  # output layer
net.setup()

# Forward pass
output = net.forward(X , y)

print("Network output probabilities:", output)


4
5
3
5
3
5
3
5


TypeError: Layer.forward() missing 1 required positional argument: 'input'

In [32]:
input = [1,2,3]
layer1 = Layer(2, "sigmoid")
layer2 = Layer(5 , "sigmoid")
layer3 = Layer(2 , "sigmoid")
softmax_layer1 = softmax_layer()
layer1.setup(3)
layer2.setup(2)
layer3.setup(5)
output = layer1.forward(input)
output = layer2.forward(output)
output = layer3.forward(output)
output = softmax_layer1.forward(output)

In [33]:
print(output)

[[0.51995055 0.48004945]]


In [39]:
ce = Cross_Entropy_layer()


In [8]:
a = layer3.backward(output, 0.01)
a=layer2.backward(a, 0.01)


In [43]:
print(ce.forward(output, np.array([1,0])))

0.6540215707169154


In [ ]:
class Network():


  def __init__(self):
    self.layers = []
    self.loss = None
    self.in_dim = None
    self.out_dim = None

  def add(self, layer):
    self.layers.append(layer)

  def forward(self, input):
    self.layers[0].input = input
    for i in range(1, len(self.layers)):
      self.layers[i].input = self.layers[i-1].output
      self.layers[i].forward()
    self.layers[-1].output = self.layers[-1].input
    return self.layers[-1].output


In [10]:
a = np.tanh

In [11]:
a(4)

np.float64(0.999329299739067)